In [4]:
import sqlite3
import pandas as pd
import os

In [3]:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html

In [5]:
path = os.getcwd()+'/'

In [43]:
data = pd.read_csv(path+'card_transactions.csv')

In [44]:
data.shape

(96708, 10)

In [45]:
data.columns

Index(['Recordnum', 'Cardnum', 'Date', 'Merchantnum', 'Merch Description',
       'Merchant State', 'Merchant Zip', 'Transtype', 'Amount', 'Fraud'],
      dtype='object')

In [46]:
data.rename(columns={"Merch Description": "Merch_Description", 
                      "Merchant State": "Merchant_State",
                      'Merchant Zip': 'Merchant_Zip'},
           inplace = True)

In [47]:
data.columns

Index(['Recordnum', 'Cardnum', 'Date', 'Merchantnum', 'Merch_Description',
       'Merchant_State', 'Merchant_Zip', 'Transtype', 'Amount', 'Fraud'],
      dtype='object')

## create sqlite files and tables

In [48]:
conn = sqlite3.connect(path+'fraud.sqlite')

In [49]:
data.to_sql(name='card_transactions', con = conn, index = False, chunksize = None)

In [54]:
sql_code = '''
select * from card_transactions limit 0,3;
'''

In [52]:
sql_code = '''
select Merch_Description from card_transactions limit 0,3;
'''

## use pandas built-in function

In [55]:
pd.read_sql_query(sql_code, conn)

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0


## use cursor

In [36]:
c = conn.cursor()

In [39]:
c.execute(sql_code)
result = c.fetchall()

In [30]:
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0


## define function for convenience

In [56]:
def pysql(sqlcode):
    return pd.read_sql_query(sqlcode, conn)

In [57]:
pysql('select * from card_transactions limit 3,5;')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud
0,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
1,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
2,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
3,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0


In [59]:
pysql('select count(distinct Merch_Description) as unique_desciptions from card_transactions ;')

,unique_desciptions
0,13125
